# 머신러닝 with Python 
## 09 앙상블 학습 보팅 실습

##### 보팅 알고리즘을 활용해 꽃 데이터를 분류하는 모형을 생성해본다.

### 데이터 불러오기

In [1]:
from sklearn import datasets 

In [2]:
raw_iris = datasets.load_iris() # 꽃 데이터 가져오기

In [4]:
# 데이터 살펴보기

raw_iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

### 피처, 타깃 데이터 지정

보통 train data는 행렬이므로 대문자 X, 예측되어야 할 값은 스칼라로 소문자 y가 된다는 것을 들은게 생각이 나부렀다.}

In [5]:
X = raw_iris.data
y = raw_iris.target

### 트레이닝/테스트 데이터 분할

In [7]:
from sklearn.model_selection import train_test_split              # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 분리. randomstate는 고정

### 데이터 표준화

In [8]:
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                     # 표준화 스케일러 지정
std_scale.fit(X_tn)                              # 트레이닝 피처를 기준으로 표준화를 적합시키기


X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)             # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습 - 보팅 학습

이번 실습에서는 로지스틱 회귀분석, 가우시안 나이브 베이즈, 서포트 벡터 머신을 사용한다.

In [12]:
from sklearn.linear_model import LogisticRegression # 로지트틱 회귀분석
from sklearn import svm                             # support vector machine
from sklearn.naive_bayes import GaussianNB          # Gussian Naive Bayes
from sklearn.ensemble import VotingClassifier       # 분류 문제이기 때문이 VotingClassifier를 사용. 회귀문제라면 VotingRegressor를 사용한다.

clf1 = LogisticRegression(multi_class = 'multinomial', 
                          random_state = 1)         # 첫 번째 모형을 로지스틱 회귀분석이라고 정함
clf2 = svm.SVC(kernel = 'linear', 
              random_state = 1)                     # 두 번째 모형을 서포트 벡터 머신 모형이라고 정함
clf3 = GaussianNB()                                 # 세 번째 모형은 가우시안 나이브 베이즈

clf_voting = VotingClassifier(estimators = [        # estimators : 미리 만든 세 가지 모형을 의미
                                    ('lr', clf1),
                                    ('svm', clf2),
                                    ('gnb', clf3),
                                                ],
                             voting = 'hard',       # voting : hard(default)=과반수가 넘는 라벨, soft=확률이 가장 높은 라벨
                             weights = [1, 1, 1])   # weights : 각 모형의 비율. 중요도 가중치를 어떻게 줄지인 것으로 예상됨. 

clf_voting.fit(X_tn_std, y_tn)                      # 만들어진 모형에 표준화된 트레이닝 피처 데이터와 트레이닝 타깃 데이터를 넣고 적합시킨다.

VotingClassifier(estimators=[('lr',
                              LogisticRegression(multi_class='multinomial',
                                                 random_state=1)),
                             ('svm', SVC(kernel='linear', random_state=1)),
                             ('gnb', GaussianNB())],
                 weights=[1, 1, 1])